# Welcome to the RealityEngines.AI Recommendations Engine Workbook! 
This workbook provides you with a hands on environment to build a Recommendations Model using the RealityEngines.AI Python Client Library.

## Prerequisites

- [Sign up](https://realityengines.ai/app/signup) for RealityEngines.AI
- Navigate to the [API Keys Dashboard](https://realityengines.ai/app/profile/apikey) and generate an API key


---





## Set up your development environment



1. Install the RealityEngines.AI library with the command below


In [0]:
!pip install realityengines 

2. Add your RealityEngines.AI [API Key](https://realityengines.ai/app/profile/apikey).

In [0]:
#@title RealityEngines.AI API Key

api_key = ''  #@param {type: "string"}

3. Import the RealityEngines.AI library and instatiate a client.

In [0]:
from realityengines.client import ReClient

In [0]:
client = ReClient(api_key)

## 1. Create a Project

Projects are the basic containers in RealityEngines.AI. By providing a business use case, we tailor our deep learning algorithms to produce the best performing model possible.

The command below will create a project with the name, "Example Movie Recommendations" and assigned the use case, "USER_RECOMMENDATION." You use the optional command below to see all available use cases.

In [0]:
project = client.create_project(name='Example Movie Recommendations', use_case='USER_RECOMMENDATIONS')
project.to_dict()

### (Optional) See all available use cases

In [0]:
client.list_use_cases()

## 2. Import Datasets to your Project

Just like RealityEngines.AI Projects have a type, we also have special dataset types to help declare the purpose of each file.

For this workbook we have provided a few smaller datasets to teach you how to import datasets into a project.

Links to the sample datasets used in this example:

- [movies_small.csv](https://github.com/jerry-realityengines/test_repo/blob/master/movies_small.csv)
- [ratings_small.csv](https://github.com/jerry-realityengines/test_repo/blob/master/ratings_small.csv)


You can use the commands below to check what kind of datasets are required for your use case.

In [0]:
#@title RealityEngines.AI Use Case

use_case = 'USER_RECOMMENDATIONS'  #@param {type: "string"}

In [0]:
client.describe_use_case_requirements(use_case)

The next set of commands allows you to upload and attach your datasets to your project. 

In [0]:
#@title Dataset Download Location

# Change this value if the datasets are in a different directory from this notebook's context
DOWNLOAD_LOCATION = ''  #@param {type: "string"}

In [0]:
import os
interactions_upload = client.create_dataset_upload('Movie Ratings', project_id=project.project_id, project_dataset_type='USER_ITEM_INTERACTIONS')
with open(os.path.join(DOWNLOAD_LOCATION, 'ratings_small.csv')) as file:
    interactions_dataset = interactions_upload.upload_file(file)

items_upload = client.create_dataset_upload('Movie Infos', project_id=project.project_id, project_dataset_type='ITEM_ATTRIBUTES')
with open(os.path.join(DOWNLOAD_LOCATION, 'movies_small.csv')) as file:
    items_dataset = items_upload.upload_file(file)

uploaded_datasets = [interactions_dataset, items_dataset]

The command below is a blocking call for each dataset; RealityEngines.AI is processing and inspecting the datasets to automatically detect the schema.

In [0]:
for dataset in uploaded_datasets:
    dataset.wait_for_inspection()

To see how RealityEngines.AI detected the dataset schemas, we can list the project's datasets and inspect the schema:

In [0]:
project_datasets = project.list_datasets()
project_datasets[0].project_dataset_schema.to_dict()

# 3. Train a Model

We've automatically detected the schema for your datasets, identified the ID columns and joined the metadata on top of the ratings datasets. We're ready to train!

To be sure, let's call `Validate Datasets` to confirm that we have all the required datasets and each of our datasets have all the required columns.

In [0]:
project.validate_datasets().valid

In [0]:
model = project.train_model()

After we start training the model, we can call this blocking call that continually checks the status of the model until it is done training and evaluating.

In [0]:
model.wait_for_evaluation()

### Evaluate your Model Metrics

The last step after model training is model evaluation, where various metrics are generated that allow you to get an idea of the quality of the model.

In [0]:
model.get_metrics()

# 4. Deploy Model

After we have successfully trained our model with our datasets, we need to create a deployment to be able to use our model to perform predictions. Each deployment makes the trained model available for prediction requests from both users and other software systems.


We can use the command below to deploy the model we just trained.

In [0]:
deployment = model.create_deployment('My First Model Deployment')
deployment.wait_for_deployment()  # It can take a minute or two for a deployment to be ready

After we deploy our model, we need to create a deployment token for authentication. This token is only authorized to predict on deployments in this project, so it's safe to embed this model inside of an application or website. 


In [0]:
deployment_token = project.create_deployment_token()
deployment_token

## 5. Predict

Once the deployment is Active and you have a deployment token for this project, you can call the prediction request command below. The 'userId' variable assigned to 'data' is the userId for each user included in your datasets. An example userId you can use is: '1107'.

In [0]:
client.predict(deployment_id=deployment.deployment_id, auth_token= deployment_token.auth_token, data={'userId': '1107'})